In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2
import tifffile as tiff
#from tqdm import tqdm

from keras.models import Sequential, Model ,model_from_json
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input, merge, UpSampling2D, Cropping2D, ZeroPadding2D, Reshape, core, Convolution2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras import backend as K
from keras.optimizers import SGD
from keras.layers.merge import concatenate

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split

#custom
import tensorflow as tf

#from new tutorial
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img

import random as rand

In [2]:
os.getcwd()

'/mnt/hgfs/VMsharedFolder/git/final-project-landandbuildingsatimg-ccny/notebooks/scratch'

In [3]:
datapath = "/mnt/hgfs/VMsharedFolder/git/misc"

In [5]:
x_train = []
x_test = []
y_train = []

path = ""
name = "Unet"
weights_path = path + name + '.h5'

'''df_train = pd.read_csv('../input/train_v2.csv')
df_test = pd.read_csv('../input/sample_submission_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))'''

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

'''
for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('../input/test-tif-v2/{}.tif'.format(f), -1)
    x_test.append(cv2.resize(img, (img_size, img_size)))
x_test  = np.array(x_test, np.float32)/255. 

for f, tags in tqdm(df_train.values, miniters=1000):
    #https://stackoverflow.com/questions/37512119/resize-transparent-image-in-opencv-python-cv2
    #If you load a 4 channel image, the flag -1 indicates that the image is loaded unchanged, so you can load and split all 4 channels directly.
    img = cv2.imread('../input/train-tif-v2/{}.tif'.format(f), -1)#0-1 voir au dessus les 2 comments
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (img_size, img_size)))
    y_train.append(targets)
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)/255.
'''

y_path = "/processed/val/masks"
#x_train1 = cv2.imread(datapath+'/RGB-PanSharpen_AOI_2_Vegas_img991.tif' ,1)
x_train = tiff.imread(datapath+'/RGB-PanSharpen_AOI_2_Vegas_img991.tif')
x_train1 = tf.convert_to_tensor(x_train ,np.float32)
x_train2 = tf.expand_dims(x_train1, axis=-1)
y_train = np.load(datapath+y_path+'/991_mask.npy')
y_train1 = (y_train > 0).astype(np.int32)
print(x_train.shape)
print(x_train1.shape)
print(x_train2.shape)
print(y_train.shape)
print(y_train1.shape)

(650, 650, 3)
(650, 650, 3)
(650, 650, 3, 1)
(650, 650)
(650, 650)


In [6]:
print(x_train1.shape)
with pd.option_context('display.max_rows', 5
                       , 'display.max_columns', 5
                       ,'display.width', None
                      ,'display.max_colwidth', None):
    print(x_train1)

(650, 650, 3)
tf.Tensor(
[[[109. 258. 236.]
  [159. 311. 268.]
  [188. 342. 287.]
  ...
  [481. 401. 343.]
  [458. 378. 330.]
  [470. 389. 337.]]

 [[127. 277. 247.]
  [159. 311. 268.]
  [161. 310. 268.]
  ...
  [506. 426. 358.]
  [459. 378. 330.]
  [499. 419. 355.]]

 [[142. 291. 255.]
  [129. 275. 246.]
  [158. 306. 264.]
  ...
  [490. 411. 348.]
  [493. 414. 351.]
  [461. 383. 332.]]

 ...

 [[556. 802. 599.]
  [557. 806. 601.]
  [561. 810. 605.]
  ...
  [620. 853. 619.]
  [630. 868. 629.]
  [607. 845. 615.]]

 [[541. 787. 590.]
  [553. 801. 598.]
  [557. 807. 603.]
  ...
  [606. 838. 610.]
  [614. 851. 618.]
  [591. 828. 605.]]

 [[567. 816. 608.]
  [553. 799. 598.]
  [548. 796. 596.]
  ...
  [601. 834. 608.]
  [609. 845. 615.]
  [589. 826. 604.]]], shape=(650, 650, 3), dtype=float32)


In [7]:
with pd.option_context('display.max_rows', 5
                       , 'display.max_columns', 5
                       ,'display.width', None
                      ,'display.max_colwidth', None):
    print(y_train)

[[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [8]:
processed_image = "/processed/val/images"
processed_mask = "/processed/val/masks"
x_example = np.load(datapath+processed_mask+'/991_mask.npy')

In [98]:
cv2.imshow("Image", x_train)
cv2.moveWindow("Image", 0, 0)
print(x_train.shape)

k = cv2.waitKey(0) & 0xFF
if k == 27:         # wait for ESC key to exit
    cv2.destroyAllWindows()

(650, 650, 3)


In [9]:
input_dir = "images/"
target_dir = "annotations/trimaps/"
img_size = (x_train.shape[0], x_train.shape[1])
num_classes = 2
batch_size = 32

In [34]:
class OxfordPets(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)
            # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
            y[j] -= 1
        return x, y

In [89]:
from tensorflow.keras import layers


def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(img_size, num_classes)
model.summary()

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [38]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

callbacks = [
    keras.callbacks.ModelCheckpoint(datapath+"/oxford_segmentation.h5", save_best_only=True)
]


In [39]:

# Train the model, doing validation at the end of each epoch.
epochs = 15
#model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)
model.fit(x_train1, epochs=epochs, validation_data=y_train, callbacks=callbacks)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [46]:
'''X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2)   

print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_val), len(Y_val))'''

def get_crop_shape(target, refer):
        # width, the 3rd dimension
        cw = (target.get_shape()[2] - refer.get_shape()[2])
        #cw = (target.get_shape()[2] - refer.get_shape()[2]).value
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1])
        #ch = (target.get_shape()[1] - refer.get_shape()[1]).value
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)
    
def get_unet(n_ch,patch_height,patch_width):
    concat_axis = 3

    inputs = Input(shape=(patch_height, patch_width, n_ch))
    #inputs = Input((patch_height, patch_width))
    
    conv1 = Conv2D(32, (3, 3), padding="same", name="conv1_1", activation="relu", data_format="channels_last")(inputs)
    conv1 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)
    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool1)
    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)

    conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool2)
    conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv3)

    conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool3)
    conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv4)

    conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool4)
    conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv5)

    up_conv5 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv5)
    ch, cw = get_crop_shape(conv4, up_conv5)
    crop_conv4 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv4)
    up6   = concatenate([up_conv5, crop_conv4], axis=concat_axis)
    conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(up6)
    conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv6)

    up_conv6 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv6)
    ch, cw = get_crop_shape(conv3, up_conv6)
    crop_conv3 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv3)
    up7   = concatenate([up_conv6, crop_conv3], axis=concat_axis)
    conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(up7)
    conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv7)

    up_conv7 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv7)
    ch, cw = get_crop_shape(conv2, up_conv7)
    crop_conv2 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv2)
    up8   = concatenate([up_conv7, crop_conv2], axis=concat_axis)
    conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(up8)
    conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv8)

    up_conv8 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv8)
    ch, cw = get_crop_shape(conv1, up_conv8)
    crop_conv1 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv1)
    up9   = concatenate([up_conv8, crop_conv1], axis=concat_axis)
    conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(up9)
    conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv9)

    #ch, cw = get_crop_shape(inputs, conv9)
    #conv9  = ZeroPadding2D(padding=(ch[0],cw[0]), data_format="channels_last")(conv9)
    #conv10 = Conv2D(1, (1, 1), data_format="channels_last", activation="sigmoid")(conv9)
    
    flatten =  Flatten()(conv9)
    #flatten =  Flatten()(conv1)
    Dense1 = Dense(512, activation='relu')(flatten)
    BN =BatchNormalization() (Dense1)
    Dense2 = Dense(17, activation='sigmoid')(BN)
    
    model = Model(inputs=inputs, outputs=Dense2)
    
    return model

In [11]:
x_train1.shape

TensorShape([650, 650, 3])

In [12]:
img_size = 65
channels = 3 #4 for tiff, 3 for jpeg


keras.backend.clear_session()

model = get_unet(channels, img_size, img_size)
#model = get_unet(channels, x_train.shape[0], x_train.shape[1])


In [41]:
print(type(x_train))
print(x_train1.shape)

<class 'numpy.ndarray'>
(650, 650, 3)


In [37]:
print(type(x_train1))
print(x_train1.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(650, 650, 3)


In [38]:
print(type(x_train2))
print(x_train2.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(650, 10, 0, 1)


In [39]:
x_train3 = tf.expand_dims(x_train1, axis=-1)
print(type(x_train3))
print(x_train3.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(650, 650, 3, 1)


In [40]:
x_train4 = MaxPooling2D(pool_size=(65, 65), data_format="channels_last")(x_train3)
print(type(x_train4))
print(x_train4.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(650, 10, 0, 1)


In [43]:
print(x_train.shape)
print(x_train1.shape)
print(x_train2.shape)
print(y_train.shape)
print(type(y_train1))
print(y_train1.shape)

(650, 650, 3)
(650, 650, 3)
(650, 10, 0, 1)
(650, 650)
<class 'numpy.ndarray'>
(650, 650)


In [47]:
x_val = x_train1
y_val = y_train1

epochs_arr  = [   20]#,      5,      5]
learn_rates = [0.001]#, 0.0003, 0.0001]

weights_path = datapath

for learn_rate, epochs in zip(learn_rates, epochs_arr):
    if os.path.isfile(weights_path):
        print("loading existing weight for training")
        model.load_weights(weights_path)
    
    opt  = optimizers.Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer=opt,
                  metrics=['accuracy'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=1),
                 ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True, verbose=2)]

    model.fit(x = x_train1, y = y_train1, validation_data=(x_val, y_val),
          batch_size=256, verbose=2, epochs=epochs, callbacks=callbacks, shuffle=True)

if os.path.isfile(weights_path):
    model.load_weights(weights_path)

Epoch 1/20


ValueError: in user code:

    /home/busybeeze/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/busybeeze/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/busybeeze/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/busybeeze/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/busybeeze/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/busybeeze/.local/lib/python3.8/site-packages/tensorflow/py

In [2]:
model_json = model.to_json()
with open("model_weights.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("datapath/model_weights.h5")
print("Saved model to disk")

NameError: name 'model' is not defined

In [ ]:


p_val = model.predict(X_val, batch_size = 128, verbose=1)
print(fbeta_score(Y_val, np.array(p_val) > 0.2, beta=2, average='samples'))

p_test = model.predict(x_test, batch_size = 128, verbose=1)



In [ ]:


result = p_test
result = pd.DataFrame(result, columns = labels)

from tqdm import tqdm
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index))) 
    
df_test['tags'] = preds
df_test.to_csv('F:/DS-main/Kaggle-main/Planet Understanding the Amazon from Space/submission_unet.csv', index=False)
